In [17]:
# import importlib.util
import importlib
import pandas as pd
import numpy as np
import torch
import sys
import os

%load_ext autoreload
%autoreload 2

sys.path.append('../code/src/')
import utils


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### CSV creation from stations

In [18]:
# Shamat
shamat_df = utils.get_df_from_shamat_csv('../data/raw/shamat/shamat_beit_dagan.csv')
radiation_df = utils.get_df_from_radiation_csv('../data/raw/shamat/radiation_beit_dagan.csv', )
shamat_df['rad'] = radiation_df['rad']

if not os.path.exists('../data/processed_csvs/'):
    os.makedirs('../data/processed_csvs/')
shamat_df.to_csv('../data/processed_csvs/shamat.csv', index=True)

/Users/avnerran/MSc/aps_analysis/transformers/notebooks/../code/src/utils.py:162: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  merged_df = numeric_df.groupby(numeric_df.columns, axis=1).mean()


In [19]:
print(len(os.listdir('../data/raw/ep/')))
le = 0
for i in os.listdir('../data/raw/ep/'):
    print(i)
    le += 1

print(le)

33
jerusalem_bikha.xlsx
nir_galim.xlsx
tel_aviv_yefet.xlsx
bnei_atarot.xlsx
.DS_Store
karmei_yosef.xlsx
yavne.xlsx
ariel.xlsx
tel_aviv_levinsky.xlsx
modiin.xlsx
ramle_omanim.xlsx
holon.xlsx
kfar_menachem_harhava.xlsx
ahisemech.xlsx
tel_aviv_lehi.xlsx
gan_yavne.xlsx
ashdod_tsfoni.xlsx
elad.xlsx
beit_shemesh.xlsx
timorim.xlsx
bnei_darom.xlsx
beit_heshmonai.xlsx
rehovot.xlsx
kiryat_malachi.xlsx
gan_darom.xlsx
or_yehuda.xlsx
kfar_menachem.xlsx
yad_binyamin.xlsx
hevel_yavne.xlsx
yad_rambam.xlsx
ashdod_kala.xlsx
rishon_herzel.xlsx
jerusalem_atarot.xlsx
33


In [21]:
# EP
# shamat_csv = pd.read_csv('../data/processed_csvs/shamat.csv')
stations_w_f0 = 0
station_wo_f0 = 0

for file in os.listdir('../data/raw/ep/'):
    if file.endswith('.xlsx'):
        station_df = utils.get_df_from_excel_station_file(f'../data/raw/ep/{file}')

        # adding the factor0 to the csv:
        try:
            station_df['factor0'] = utils.calculate_factor0(station_df, shamat_df)
            if station_df['factor0'].notna().any():
                stations_w_f0 += 1
            else:
                station_wo_f0 += 1
            station_df.to_csv(f'../data/processed_csvs/{file[:-5]}.csv', index=True)
            
        except:
            print(f'probably no measurement for the factor calculation in station: {file[:-5]}')
            station_df.to_csv(f'../data/processed_csvs/{file[:-5]}.csv', index=True)
            
            continue

print(f'total num of station: {stations_w_f0 + station_wo_f0}')
print(f'num of stations with factor0: {stations_w_f0}')
print(f'num of stations without factor0: {station_wo_f0}')

total num of station: 32
num of stations with factor0: 13
num of stations without factor0: 19


In [ ]:
'''
 the dates of the sessions are:

4-5 May 2023
8-9 May 2023
23-24 May 2023

25-26 August 2023
28-29 August 2023
 '''

'\n the dates of the sessions are:\n\n4-5 May 2023\n8-9 May 2023\n23-24 May 2023\n\n25-26 August 2023\n28-29 August 2023\n '

#### Pulling timestamps from pop tensor

In [22]:
# getting the time of the flights:
# the time in the list is the time at the center of the flight
flight_times = torch.load('../data/raw/drone/pom_timestamps.pt', map_location='cpu', weights_only=False)
flight_time_list = flight_times.to_list()

#### Saving the stations as tensors

In [37]:
if not os.path.exists('../data/station_tensors/'):
    os.makedirs('../data/station_tensors/')


for station in os.listdir('../data/processed_csvs/'):
    if station.endswith('.csv'):
        t = utils.station_csv_to_tensor(f'../data/processed_csvs/{station}', flight_time_list, max_features=15)
        print(t.shape)
        torch.save(t, f'../data/station_tensors/{station[:-4]}.pt')

torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])
torch.Size([66, 5, 15])


In [ ]:
# the tensors shapes are [P, L, Fs]: [66, 5, 15] when the missing features are nan values.

#### Creating the metadata json files for each stations

In [ ]:
for station in os.listdir('../data/processed_csvs/'):
    if station.endswith('.csv'):
        station = station[:-4]
        meta = utils.save_tensor_metadata(f"../data/processed_csvs/{station}.csv", station, tensor_shape=[[66, 5, 15]] output_dir='../data/metadata/')

Now we have a tensor for each station: [P, L, Fs].  
Plus a metadata json file for each station.  

Plus, we have the tensor of the Ozone measurements: [P, h, Fd].